In [19]:
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,r2_score,mean_absolute_error,mean_squared_error
from sklearn.tree import DecisionTreeClassifier
eps = np.finfo(float).eps
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [20]:
#Assigning column names to data
df=pd.read_csv('data.csv',delimiter=";")
alpha = 0.01
iterations = 1000
threshold = 0.5

In [21]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
Y_train=train['quality']
Y_test=test['quality']
X_train= train.drop(['quality'],axis=1)
X_test= test.drop(['quality'],axis=1)

In [22]:
X_train = (X_train - X_train.mean())/X_train.std()
X_test = (X_test - X_test.mean())/X_test.std()

In [23]:
X_train = pd.concat([X_train,Y_train],axis=1)
ones = np.ones([X_train.shape[0],1])
Y_train = X_train.iloc[:,11:12].values
X_train = X_train.iloc[:,0:11]
X_train = np.concatenate((ones,X_train),axis=1)

In [24]:
theta = np.zeros([15,12])

In [25]:
def G(x):
    p=1+(np.exp(-x))
    return 1/p

In [26]:
def gradient_decent(X_train,Y_train,theta1,learning_rate,iterations):
    for i in range(iterations):
        h = G(np.dot(X_train,theta1.T))
        theta1 = theta1 - (learning_rate/len(X_train)) * np.sum(X_train * (h - Y_train), axis=0)
    return theta1

In [27]:
x=0
for i in range(4,9):
    for j in range(i+1,10):
        theta1 = np.zeros([1,12])
        
        Xcopy = []
        Ycopy = []
        W = np.array(Y_train)
        for k in range(len(W)):
            if W[k] == j or W[k]==i:
                Xcopy.append(X_train[k])
                if W[k] == i:
                    Ycopy.append(1)
                else:
                    Ycopy.append(0)
        Xcopy = np.array(Xcopy)
        Ycopy = np.array(Ycopy)
        Ycopy = Ycopy.reshape((len(Ycopy),1))

        theta[x]=gradient_decent(Xcopy,Ycopy,theta1,alpha,iterations)
        x = x + 1

In [28]:
y_pred = []
for index,rows in X_test.iterrows():
    rows = list(rows)
    counts = {}
    label = 0
    for i in range(4,10):
        counts[i]=0
    max_h = 0
    c = 0
    for a in range(4,9):
        for b in range(a+1,10):
            y = 0
            for i in range(len(rows)):
                y = y + rows[i]*theta[c][i+1]
            y = y + theta[c][0]
            y = G(y)
            c = c + 1
            if y >= threshold:
                counts[a]=counts[a]+1
            else:
                counts[b]=counts[b]+1
    for i in range(4,10):
        if(counts[i]>=max_h):
            max_h=counts[i]
            label=i
    y_pred.append(label)

In [29]:
print((y_pred == Y_test).mean())
print(accuracy_score(Y_test, y_pred))

0.5541176470588235
0.5541176470588235
